# Detecting temperature targets
##### authors: Elizabeth A. Barnes and Noah Diffenbaugh

##### README:

* All user-defined parameters (other than plotting colors etc) are defined in the top cell called "User-defined Experiment Parameters". Most of your edits should just be there.
* The code will train either a typical regression problem ```reg``` or it will predict the conditional normal distribution ```shash2```
* The code provides the option of saving the tensorflow models (and their metadata) so that you can load them later and make predictions. This may be particularly useful if you find a good model that you like (and don't want to retrain over and over again) or if you want to save ~50 models and then use them to predict the observational fields to make a distribution of uncertainty (like I did in the google slides)
* The random seed calls should be written so that everything is exactly reproducible. So, run the same code tomorrow and you should get identical answers.


## User-defined Experiment Parameters

In [1]:
exp_settings = {
    "exp_name": 'test',
    "save_model": True,
    "n_models": 1,                        # the number of networks you want to train
    "ssp" : "370",                        #[options: '126' or '370']
    "target_temp": 1.5,
    "n_train_val_test" : (7,3,0),
    "baseline_yr_bounds": (1850,1899),
    "training_yr_bounds": (1970,2100),
    "anomaly_yr_bounds": (1951,1980),
    "anomalies": True,                    #[options: True or False]
    "remove_map_mean": False,             #[options: False or "weighted" or "raw"]
    "show_plots" : False,                 #[options: False or True]

    "network_type": 'reg',                #[options: "reg" or "shash2"]
    "hiddens": [10,10],
    "dropout_rate": 0.,
    "ridge_param": [1.,0.0], 
    "learning_rate": 0.00001,
    "batch_size": 64,
    "rng_seed": 8889,
    "act_fun": ["relu","relu"],
    "n_epochs": 25_000,
    "patience": 50,
    "verbosity": 0,
}

## Python setup

In [2]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
print('IN_COLAB = ' + str(IN_COLAB))

IN_COLAB = True


In [3]:
if IN_COLAB:
    !pip install xarray==0.20.2
    !pip install nc-time-axis

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout, Softmax
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow import keras

import matplotlib as mpl
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["figure.dpi"] = 150
savefig_dpi = 300
np.warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

     |████████████████████████████████| 845 kB 5.0 MB/s 
  Attempting uninstall: xarray
    Found existing installation: xarray 0.18.2
    Uninstalling xarray-0.18.2:
      Successfully uninstalled xarray-0.18.2


In [4]:
print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"xarray version = {xr.__version__}")  
print(f"tensorflow version = {tf.__version__}")  
print(f"tensorflow-probability version = {tfp.__version__}")  

python version = 3.7.12 (default, Jan 15 2022, 18:48:18) 
[GCC 7.5.0]
numpy version = 1.21.5
xarray version = 0.20.2
tensorflow version = 2.8.0
tensorflow-probability version = 0.16.0


## Data functions

In [5]:
DATA_DIRECTORY = 'data/'

def save_tf_model(model, model_name):
    
    import json
    
    if IN_COLAB:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        model_path = './Colab Notebooks/'
        raise ValueError('@Noah - please set the GoogleDrive model_path above where you want to save things')
    else:
        MODEL_DIRECTORY = 'saved_models/'        
        
    tf.keras.models.save_model(
        model, MODEL_DIRECTORY + model_name + "_model", overwrite=True
    )
    with open(MODEL_DIRECTORY + model_name + '_metadata.json', 'w') as json_file:
        json_file.write(json.dumps(exp_settings))

def get_netcdf_da(filename):
    if IN_COLAB:
        !pip install wget
        import wget
        nc_file = wget.download("https://eabarnes-data.atmos.colostate.edu/share/" + filename)
    else:
        nc_file = DATA_DIRECTORY + filename
    
    da = xr.open_dataarray(nc_file)
    return da

In [6]:
if exp_settings["ssp"] == '370':
    filenames = ('tas_Amon_historical_ssp370_CanESM5_r1-10_ncecat_ann_mean_2pt5degree.nc',
                 'tas_Amon_historical_ssp370_GISS-E2-1-G_r1-10_ncecat_ann_mean_2pt5degree.nc',
                 'tas_Amon_historical_ssp370_IPSL-CM6A-LR_r1-10_ncecat_ann_mean_2pt5degree.nc',
                 'tas_Amon_historical_ssp370_UKESM1-0-LL_r1-10_ncecat_ann_mean_2pt5degree.nc',
                 'tas_Amon_historical_ssp370_ACCESS-ESM1-5_r1-10_ncecat_ann_mean_2pt5degree.nc',
                )
elif exp_settings["ssp"] == '126':
    filenames = ('tas_Amon_historical_ssp126_CanESM5_r1-10_ncecat_ann_mean_2pt5degree.nc',
                 'tas_Amon_historical_ssp126_MIROC6_r1-10_ncecat_ann_mean_2pt5degree.nc',
                 'tas_Amon_historical_ssp126_ACCESS-ESM1-5_r1-10_ncecat_ann_mean_2pt5degree.nc',
                 'tas_Amon_historical_ssp126_UKESM1-0-LL_r1-10_ncecat_ann_mean_2pt5degree.nc',
            )
else:
    raise NotImplementedError('no such SSP')

N_GCMS = len(filenames)


### Define the target year, labels and pre-processing

For starters, we will define the target year as the year that the ensemble mean, global mean temperature crosses the ```TARGET_TEMP``` threshold.

In [7]:
def get_labels(da, plot=False):
    # compute the ensemble mean, global mean temperature
    # these computations should be based on the training set only
    da_ens = da.mean(axis=0)
    weights = np.cos(np.deg2rad(da_ens.lat))
    weights.name = "weights"
    temp_weighted = da_ens.weighted(weights)
    global_mean = temp_weighted.mean(("lon", "lat"))
    
    global_mean_ens = da.weighted(weights)
    global_mean_ens = global_mean_ens.mean(("lon","lat"))
    
    # compute the target year 
    baseline_mean = global_mean.sel(time=slice(str(exp_settings["baseline_yr_bounds"][0]),str(exp_settings["baseline_yr_bounds"][1]))).mean('time')
    iwarmer = np.where(global_mean.values > baseline_mean.values+exp_settings["target_temp"])[0]
    target_year = global_mean["time"].values[iwarmer[0]]

    # plot the calculation to make sure things make sense
    if plot == True:
        for ens in np.arange(0,global_mean_ens.shape[0]):
            global_mean_ens[ens,:].plot(linewidth=1.0,color="gray",alpha=.5)
        global_mean.plot(linewidth=2,label='data',color="aqua")
        plt.axhline(y=baseline_mean, color='k', linestyle='-', label='baseline temp')
        plt.axhline(y=baseline_mean+exp_settings["target_temp"], color='tab:blue',linewidth=1., linestyle='--', label='target temp')
        plt.axvline(x=target_year,color='tab:blue',linewidth=1., linestyle='--', label='target year')
        global_mean_obs.plot(linewidth=2,label='data',color="tab:orange")        
        plt.xlabel('year')
        plt.ylabel('temp (K)')
        plt.title(f + '\ntargets [' + str(target_year.year) + ', ' + str(exp_settings["target_temp"]) + 'C]',
                  fontsize = 8,
                 )
        plt.show()
    
    # define the labels
    print('TARGET_YEAR = ' + str(target_year.year))
    labels = target_year.year - da['time.year'].values
    
    return labels, da['time.year'].values, target_year.year

def preprocess_data(da, MEMBERS):

    if MEMBERS is None:
        new_data = da
    else:
        new_data = da[MEMBERS,:,:,:]

    if exp_settings["anomalies"] is True:
        new_data = new_data - new_data.sel(time=slice(str(exp_settings["anomaly_yr_bounds"][0]),str(exp_settings["anomaly_yr_bounds"][1]))).mean('time')
        
    if exp_settings["remove_map_mean"]  == 'raw':
        new_data = new_data - new_data.mean(("lon","lat"))
    elif exp_settings["remove_map_mean"] == 'weighted':
        weights = np.cos(np.deg2rad(new_data.lat))
        weights.name = "weights"
        new_data_weighted = new_data.weighted(weights)
        new_data = new_data - new_data_weighted.mean(("lon","lat"))

    return new_data

def make_data_split(da, data, f_labels, f_years, labels, years, MEMBERS):

    # process the data, i.e. compute anomalies, subtract the mean, etc.
    new_data = preprocess_data(da, MEMBERS)    
    
    # only train on certain samples
    iyears = np.where((f_years >= exp_settings["training_yr_bounds"][0]) & (f_years <= exp_settings["training_yr_bounds"][1]))[0]    
    f_years = f_years[iyears]
    f_labels = f_labels[iyears]            
    new_data = new_data[:,iyears,:,:]
    
    if data is None:
        data = new_data.values
        labels = np.tile(f_labels,(len(MEMBERS),1))        
        years = np.tile(f_years,(len(MEMBERS),1))
    else:
        data = np.concatenate((data,new_data.values),axis=0)        
        labels = np.concatenate((labels,np.tile(f_labels,(len(MEMBERS),1))),axis=0)        
        years = np.concatenate((years,np.tile(f_years,(len(MEMBERS),1))),axis=0)
    
    return data, labels, years

### Load the data

#### Load observations

In [8]:
nc_filename_obs = 'Land_and_Ocean_LatLong1_185001_202112_ann_mean_2pt5degree.nc'
da_obs = get_netcdf_da(nc_filename_obs)
weights = np.cos(np.deg2rad(da_obs.lat))
weights.name = "weights"
temp_weighted = da_obs.weighted(weights)
global_mean_obs = temp_weighted.mean(("lon", "lat"), skipna=False)

data_obs = preprocess_data(da_obs, MEMBERS=None,) 
x_obs = data_obs.values.reshape((data_obs.shape[0],data_obs.shape[1]*data_obs.shape[2]))
print('np.shape(x_obs) = ' + str(np.shape(x_obs)))

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=6548f0a9c922d92a9c0cbb6cb2fe9e625f8e9d39f084b94fe3d3ccd964e3b185
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
np.shape(x_obs) = (172, 10368)


#### Load CMIP

In [9]:
N_TRAIN = exp_settings["n_train_val_test"][0]
N_VAL   = exp_settings["n_train_val_test"][1]
N_TEST  = exp_settings["n_train_val_test"][2]
ALL_MEMBERS = np.arange(0,N_TRAIN+N_VAL+N_TEST)

def get_cmip_data():
    data_train, data_val, data_test = None, None, None
    labels_train, labels_val, labels_test = None, None, None
    years_train, years_val, years_test = None, None, None
    target_years = []

    for f in filenames:
        print(f)
        da = get_netcdf_da(f)
        f_labels, f_years, f_target_year = get_labels(da, plot=exp_settings["show_plots"])

        # create sets of train / validaton / test
        target_years = np.append(target_years,f_target_year)
        data_train, labels_train, years_train = make_data_split(da, 
                                                                data_train, 
                                                                f_labels, 
                                                                f_years, 
                                                                labels_train,
                                                                years_train,
                                                                train_members)
        data_val, labels_val, years_val       = make_data_split(da, 
                                                                data_val, 
                                                                f_labels, 
                                                                f_years, 
                                                                labels_val,
                                                                years_val,
                                                                val_members)
        data_test, labels_test, years_test    = make_data_split(da, 
                                                                data_test, 
                                                                f_labels, 
                                                                f_years, 
                                                                labels_test,
                                                                years_test,
                                                                test_members)

    print('---------------------------')        
    YEARS_UNIQUE = np.unique(years_train)
    print('data_train.shape = ' + str(np.shape(data_train)))
    print('data_val.shape = ' + str(np.shape(data_val)))
    print('data_test.shape = ' + str(np.shape(data_test)))
    
    x_train = data_train.reshape((data_train.shape[0]*data_train.shape[1],data_train.shape[2]*data_train.shape[3]))
    x_val   = data_val.reshape((data_val.shape[0]*data_val.shape[1],data_val.shape[2]*data_val.shape[3]))
    x_test  = data_test.reshape((data_test.shape[0]*data_test.shape[1],data_test.shape[2]*data_test.shape[3]))

    y_train = labels_train.reshape((data_train.shape[0]*data_train.shape[1],))
    y_val   = labels_val.reshape((data_val.shape[0]*data_val.shape[1],))
    y_test  = labels_test.reshape((data_test.shape[0]*data_test.shape[1],))

    y_yrs_train = years_train.reshape((data_train.shape[0]*data_train.shape[1],))
    y_yrs_val   = years_val.reshape((data_val.shape[0]*data_val.shape[1],))
    y_yrs_test  = years_test.reshape((data_test.shape[0]*data_test.shape[1],))

    print(x_train.shape, y_train.shape, y_yrs_train.shape)
    print(x_val.shape, y_val.shape, y_yrs_val.shape)
    print(x_test.shape, y_test.shape, y_yrs_test.shape)  
    
    map_shape = np.shape(data_train)[2:]
    
    return x_train, x_val, x_test, y_train, y_val, y_test, y_yrs_train, y_yrs_val, y_yrs_test, target_years, map_shape

## Network and XAI functions

In [10]:
# define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                   patience=exp_settings['patience'],
                                                   verbose=1,
                                                   mode='auto',
                                                   restore_best_weights=True)


In [11]:
class Exponentiate(keras.layers.Layer):
    """Custom layer to exp the sigma and tau estimates inline."""

    def __init__(self, **kwargs):
        super(Exponentiate, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.math.exp(inputs)

class InterquartileCapture(tf.keras.metrics.Metric):
    """Compute the fraction of true values between the 25 and 75 percentiles.

    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.count = self.add_weight("count", initializer="zeros")
        self.total = self.add_weight("total", initializer="zeros")

    def update_state(self, y_true, pred, sample_weight=None):
        mu = pred[:, 0]
        sigma = pred[:, 1]

        if pred.shape[1] >= 3:
            gamma = pred[:, 2]
        else:
            gamma = tf.zeros_like(mu)

        if pred.shape[1] >= 4:
            tau = pred[:, 3]
        else:
            tau = tf.ones_like(mu)

        lower = shash_quantile(0.25, mu, sigma, gamma, tau)
        upper = shash_quantile(0.75, mu, sigma, gamma, tau)

        batch_count = tf.reduce_sum(
            tf.cast(
                tf.math.logical_and(
                    tf.math.greater(y_true[:, 0], lower),
                    tf.math.less(y_true[:, 0], upper)
                ),
                tf.float32
            )

        )
        batch_total = len(y_true[:, 0])

        self.count.assign_add(tf.cast(batch_count, tf.float32))
        self.total.assign_add(tf.cast(batch_total, tf.float32))

    def result(self):
        return self.count / self.total

    def get_config(self):
        base_config = super().get_config()
        return {**base_config}


class SignTest(tf.keras.metrics.Metric):
    """Compute the fraction of true values above the median.

    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.count = self.add_weight("count", initializer="zeros")
        self.total = self.add_weight("total", initializer="zeros")

    def update_state(self, y_true, pred, sample_weight=None):
        mu = pred[:, 0]
        sigma = pred[:, 1]

        if pred.shape[1] >= 3:
            gamma = pred[:, 2]
        else:
            gamma = tf.zeros_like(mu)

        if pred.shape[1] >= 4:
            tau = pred[:, 3]
        else:
            tau = tf.ones_like(mu)

        median = shash_median(mu, sigma, gamma, tau)

        batch_count = tf.reduce_sum(
            tf.cast(tf.math.greater(y_true[:, 0], median), tf.float32)
        )
        batch_total = len(y_true[:, 0])

        self.count.assign_add(tf.cast(batch_count, tf.float32))
        self.total.assign_add(tf.cast(batch_total, tf.float32))

    def result(self):
        return self.count / self.total

    def get_config(self):
        base_config = super().get_config()
        return {**base_config}

    
class CustomMAE(tf.keras.metrics.Metric):
    """Compute the prediction mean absolute error.

    The "predicted value" is the median of the conditional distribution.

    Notes
    -----
    * The computation is done by maintaining running sums of total predictions
        and correct predictions made across all batches in an epoch. The
        running sums are reset at the end of each epoch.

    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.error = self.add_weight("error", initializer="zeros")
        self.total = self.add_weight("total", initializer="zeros")

    def update_state(self, y_true, pred, sample_weight=None):
        mu = pred[:, 0]
        sigma = pred[:, 1]

        if pred.shape[1] >= 3:
            gamma = pred[:, 2]
        else:
            gamma = tf.zeros_like(mu)

        if pred.shape[1] >= 4:
            tau = pred[:, 3]
        else:
            tau = tf.ones_like(mu)

        predictions = shash_median(mu, sigma, gamma, tau)

        error = tf.math.abs(y_true[:, 0] - predictions)
        batch_error = tf.reduce_sum(error)
        batch_total = tf.math.count_nonzero(error)

        self.error.assign_add(tf.cast(batch_error, tf.float32))
        self.total.assign_add(tf.cast(batch_total, tf.float32))

    def result(self):
        return self.error / self.total

    def get_config(self):
        base_config = super().get_config()
        return {**base_config}
   
    
def compute_NLL(y, distr): 
    return -distr.log_prob(y) 

def compute_shash_NLL(y_true, pred):
    """Negative log-likelihood loss using the sinh-arcsinh normal distribution.

    Arguments
    ---------
    y_true : tensor
        The ground truth values.
        shape = [batch_size, n_parameter]

    pred :
        The predicted local conditionsal distribution parameter values.
        shape = [batch_size, n_parameters]

    Returns
    -------
    loss : tensor, shape = [1, 1]
        The average negative log-likelihood of the batch using the predicted
        conditional distribution parameters.

    Notes
    -----
    * The value of n_parameters depends on the chosen form of the conditional
        sinh-arcsinh normal distribution.
            shash2 -> n_parameter = 2, i.e. mu, sigma
            shash3 -> n_parameter = 3, i.e. mu, sigma, gamma
            shash4 -> n_parameter = 4, i.e. mu, sigma, gamma, tau

    * Since sigma and tau must be strictly positive, the network learns the
        log of these two parameters.

    * If gamma is not learned (i.e. shash2), they are set to 0.

    * If tau is not learned (i.e. shash2 or shash3), they are set to 1.

    """
    mu = pred[:, 0]
    sigma = pred[:, 1]

    if pred.shape[1] >= 3:
        gamma = pred[:, 2]
    else:
        gamma = tf.zeros_like(mu)

    if pred.shape[1] >= 4:
        tau = pred[:, 3]
    else:
        tau = tf.ones_like(mu)

    loss = -shash_log_prob(y_true[:, 0], mu, sigma, gamma, tau)
    return tf.reduce_mean(loss, axis=-1)

"""sinh-arcsinh normal distribution w/o using tensorflow_probability.

Functions
---------
cdf(x, mu, sigma, gamma, tau=None)
    cumulative distribution function (cdf).

log_prob(x, mu, sigma, gamma, tau=None)
    log of the probability density function.

mean(mu, sigma, gamma, tau=None)
    distribution mean.

median(mu, sigma, gamma, tau=None)
    distribution median.

prob(x, mu, sigma, gamma, tau=None)
    probability density function (pdf).

quantile(pr, mu, sigma, gamma, tau=None)
    inverse cumulative distribution function.

rvs(mu, sigma, gamma, tau=None, size=1)
    generate random variates.

stddev(mu, sigma, gamma, tau=None)
    distribution standard deviation.

variance(mu, sigma, gamma, tau=None)
    distribution variance.

Notes
-----
* This module uses only tensorflow.  This module does not use the
tensorflow_probability library.

* The sinh-arcsinh normal distribution was defined in [1]. A more accessible
presentation is given in [2]. 

* The notation and formulation used in this code was taken from [3], page 143.
In the gamlss.dist/CRAN package the distribution is called SHASHo. 

* There is a typographical error in the presentation of the probability 
density function on page 143 of [3]. There is an extra "2" in the denomenator
preceeding the "sqrt{1 + z^2}" term.

References
----------
[1] Jones, M. C. & Pewsey, A., Sinh-arcsinh distributions,
Biometrika, Oxford University Press, 2009, 96, 761-780.
DOI: 10.1093/biomet/asp053.

[2] Jones, C. & Pewsey, A., The sinh-arcsinh normal distribution,
Significance, Wiley, 2019, 16, 6-7.
DOI: 10.1111/j.1740-9713.2019.01245.x.
https://rss.onlinelibrary.wiley.com/doi/10.1111/j.1740-9713.2019.01245.x

[3] Stasinopoulos, Mikis, et al. (2021), Distributions for Generalized 
Additive Models for Location Scale and Shape, CRAN Package.
https://cran.r-project.org/web/packages/gamlss.dist/gamlss.dist.pdf

"""
import numpy as np
import scipy
import scipy.stats
import tensorflow as tf

__author__ = "Randal J. Barnes and Elizabeth A. Barnes"
__date__ = "14 January 2022"


SQRT_TWO = 1.4142135623730950488016887
ONE_OVER_SQRT_TWO = 0.7071067811865475244008444
TWO_PI = 6.2831853071795864769252868
SQRT_TWO_PI = 2.5066282746310005024157653
ONE_OVER_SQRT_TWO_PI = 0.3989422804014326779399461


def _jones_pewsey_P(q):
    """P_q function from page 764 of [1].

    Arguments
    ---------
    q : float, array like

    Returns
    -------
    P_q : array like of same shape as q.

    Notes
    -----
    * The formal equation is

            jp = 0.25612601391340369863537463 * (
                scipy.special.kv((q + 1) / 2, 0.25) +
                scipy.special.kv((q - 1) / 2, 0.25)
            )

        The strange constant 0.25612... is "sqrt( sqrt(e) / (8*pi) )" computed
        with a high-precision calculator.  The special function

            scipy.special.kv

        is the Modified Bessel function of the second kind: K(nu, x).

    * But, we cannot use the scipy.special.kv function during tensorflow
        training.  This code uses a 6th order polynomial approximation in
        place of the formal function.

    * This approximation is well behaved for 0 <= q <= 10. Since q = 1/tau
        or q = 2/tau in our applications, the approximation is well behaved
        for 1/10 <= tau < infty.

    """
    # A 6th order polynomial approximation of log(_jones_pewsey_P) for the
    # range 0 <= q <= 10.  Over this range, the max |error|/true < 0.0025.
    # These coefficients were computed by minimizing the maximum relative
    # error, and not by a simple least squares regression.
    coeffs = [
        9.37541380598926e-06,
        -0.000377732651131894,
        0.00642826706073389,
        -0.061281078712518,
        0.390956214318641,
        -0.0337884356755193,
        0.00248824801827172
    ]
    return tf.math.exp(tf.math.polyval(coeffs, q))


def shash_cdf(x, mu, sigma, gamma, tau=None):
    """Cumulative distribution function (cdf).

    Parameters
    ----------
    x : float (batch size x 1) Tensor
        The values at which to compute the probability density function.

    mu : float (batch size x 1) Tensor
        The location parameter. Must be the same shape as x.

    sigma : float (batch size x 1) Tensor
        The scale parameter. Must be strictly positive. Must be the same
        shape as x.

    gamma : float (batch size x 1) Tensor
        The skewness parameter. Must be the same shape as x.

    tau : float (batch size x 1) Tensor or None
        The tail-weight parameter. Must be strictly positive. Must be the same
        shape as x. If tau is None then the default value of tau=1 is used.

    Returns
    -------
    F : float (batch size x 1) Tensor.
        The computed cumulative probability distribution function (cdf)
        evaluated at the values of x.  F has the same shape as x.

    Notes
    -----
    * This function uses the tensorflow.math.erf function rather than the
    tensorflow_probability normal distribution functions.

    """
    y = (x - mu) / sigma    
    
    if tau is None:
        z = tf.math.sinh(tf.math.asinh(y) - gamma)
    else:
        z = tf.math.sinh(tau * tf.math.asinh(y) - gamma)

    return 0.5 * (1.0 + tf.math.erf(ONE_OVER_SQRT_TWO * z))


def shash_log_prob(x, mu, sigma, gamma, tau=None):
    """Log-probability density function.

    Parameters
    ----------
    x : float (batch size x 1) Tensor
        The values at which to compute the probability density function.

    mu : float (batch size x 1) Tensor
        The location parameter. Must be the same shape as x.

    sigma : float (batch size x 1) Tensor
        The scale parameter. Must be strictly positive. Must be the same
        shape as x.

    gamma : float (batch size x 1) Tensor
        The skewness parameter. Must be the same shape as x.

    tau : float (batch size x 1) Tensor
        The tail-weight parameter. Must be strictly positive. Must be the same
        shape as x. If tau is None then the default value of tau=1 is used.

    Returns
    -------
    f : float (batch size x 1) Tensor.
        The natural logarithm of the computed probability density function
        evaluated at the values of x.  f has the same shape as x.

    Notes
    -----
    * This function is included merely to emulate the tensorflow_probability
    distributions.

    """
    return tf.math.log(shash_prob(x, mu, sigma, gamma, tau))


def shash_mean(mu, sigma, gamma, tau=None):
    """The distribution mean.

    Arguments
    ---------
    mu : float (batch size x 1) Tensor
        The location parameter.

    sigma : float (batch size x 1) Tensor
        The scale parameter. Must be strictly positive. Must be the same
        shape as mu.

    gamma : float (batch size x 1) Tensor
        The skewness parameter. Must be the same shape as mu.

    tau : float (batch size x 1) Tensor
        The tail-weight parameter. Must be strictly positive. Must be the same
        shape as mu. If tau is None then the default value of tau=1 is used.

    Returns
    -------
    x : float (batch size x 1) Tensor.
        The computed distribution mean values.

    Notes
    -----
    * This equation for evX can be found on page 764 of [1].

    """
    if tau is None:
        evX = tf.math.sinh(gamma) * 1.35453080648132  
    else:
        evX = tf.math.sinh(gamma / tau) * _jones_pewsey_P(1.0 / tau)

    return mu + sigma * evX


def shash_median(mu, sigma, gamma, tau=None):
    """The distribution median.

    Arguments
    ---------
    mu : float (batch size x 1) Tensor
        The location parameter.

    sigma : float (batch size x 1) Tensor
        The scale parameter. Must be strictly positive. Must be the same
        shape as mu.

    gamma : float (batch size x 1) Tensor
        The skewness parameter. Must be the same shape as mu.

    tau : float (batch size x 1) Tensor
        The tail-weight parameter. Must be strictly positive. Must be the same
        shape as mu. If tau is None then the default value of tau=1 is used.

    Returns
    -------
    x : float (batch size x 1) Tensor.
        The computed distribution mean values.

    Notes
    -----
    * This code uses the basic formula:

        E(a*X + b) = a*E(X) + b

    * The E(X) is computed using the moment equation given on page 764 of [1].

    """
    if tau is None:
        return mu + sigma * tf.math.sinh(gamma)
    else:
        return mu + sigma * tf.math.sinh(gamma / tau)


def shash_prob(x, mu, sigma, gamma, tau=None):
    """Probability density function (pdf).

    Parameters
    ----------
    x : float (batch size x 1) Tensor
        The values at which to compute the probability density function.

    mu : float (batch size x 1) Tensor
        The location parameter. Must be the same shape as x.

    sigma : float (batch size x 1) Tensor
        The scale parameter. Must be strictly positive. Must be the same
        shape as x.

    gamma : float (batch size x 1) Tensor
        The skewness parameter. Must be the same shape as x.

    tau : float (batch size x 1) Tensor
        The tail-weight parameter. Must be strictly positive. Must be the same
        shape as x. If tau is None then the default value of tau=1 is used.

    Returns
    -------
    f : float (batch size x 1) Tensor.
        The computed probability density function evaluated at the values of x.
        f has the same shape as x.

    Notes
    -----
    * This code uses the equations on page 143 of [3], and the associated
    notation.

    """
    y = (x - mu) / sigma
    
    if tau is None:
        rsqr = tf.math.square(tf.math.sinh(tf.math.asinh(y) - gamma))
        return (
            ONE_OVER_SQRT_TWO_PI
            / sigma
            * tf.math.sqrt((1 + rsqr) / (1 + tf.math.square(y)))
            * tf.math.exp(-rsqr / 2)
        )
        
    else:
        rsqr = tf.math.square(tf.math.sinh(tau * tf.math.asinh(y) - gamma))
        return (
            ONE_OVER_SQRT_TWO_PI
            * (tau / sigma)
            * tf.math.sqrt((1 + rsqr) / (1 + tf.math.square(y)))
            * tf.math.exp(-rsqr / 2)
        )


def shash_quantile(pr, mu, sigma, gamma, tau=None):
    """Inverse cumulative distribution function.

    Arguments
    ---------
    pr : float (batch size x 1) Tensor.
        The probabilities at which to compute the values.

    mu : float (batch size x 1) Tensor
        The location parameter. Must be the same shape as pr.

    sigma : float (batch size x 1) Tensor
        The scale parameter. Must be strictly positive. Must be the same
        shape as pr.

    gamma : float (batch size x 1) Tensor
        The skewness parameter. Must be the same shape as pr.

    tau : float (batch size x 1) Tensor
        The tail-weight parameter. Must be strictly positive. Must be the same
        shape as pr. If tau is None then the default value of tau=1 is used.

    Returns
    -------
    x : float (batch size x 1) Tensor.
        The computed values at the specified probabilities. f has the same
        shape as pr.

    """
    z = tf.math.ndtri(pr)
            
    if tau is None:
        return mu + sigma * tf.math.sinh(tf.math.asinh(z) + gamma)    
    else:
        return mu + sigma * tf.math.sinh((tf.math.asinh(z) + gamma) / tau)


def shash_rvs(mu, sigma, gamma, tau=None, size=1):
    """Generate an array of random variates.

    Arguments
    ---------
    mu : float or double scalar
        The location parameter.

    sigma : float or double scalar
        The scale parameter. Must be strictly positive.

    gamma : float or double scalar
        The skewness parameter.

    tau : float or double scalar, or None
        The tail-weight parameter. Must be strictly positive. 
        If tau is None then the default value of tau=1 is used.

    size : int or tuple of ints, default=1.
        The number of random variates.

    Returns
    -------
    x : double ndarray of size=size
        The generated random variates.

    """
    z = scipy.stats.norm.rvs(size=size)
    
    if tau is None:
        return mu + sigma * np.sinh(np.arcsinh(z) + gamma)
    else:
        return mu + sigma * np.sinh((np.arcsinh(z) + gamma) / tau)


def shash_stddev(mu, sigma, gamma, tau=None):
    """The distribution standard deviation.

    Arguments
    ---------
    mu : float (batch size x 1) Tensor
        The location parameter.

    sigma : float (batch size x 1) Tensor
        The scale parameter. Must be strictly positive. Must be the same
        shape as mu.

    gamma : float (batch size x 1) Tensor
        The skewness parameter. Must be the same shape as mu.

    tau : float (batch size x 1) Tensor
        The tail-weight parameter. Must be strictly positive. Must be the same
        shape as mu. If tau is None then the default value of tau=1 is used.

    Returns
    -------
    x : float (batch size x 1) Tensor.
        The computed distribution standard deviation values.

    """
    return tf.math.sqrt(variance(mu, sigma, gamma, tau))


def shash_variance(mu, sigma, gamma, tau=None):
    """The distribution variance.

    Arguments
    ---------
    mu : float (batch size x 1) Tensor
        The location parameter.

    sigma : float (batch size x 1) Tensor
        The scale parameter. Must be strictly positive. Must be the same
        shape as mu.

    gamma : float (batch size x 1) Tensor
        The skewness parameter. Must be the same shape as mu.

    tau : float (batch size x 1) Tensor
        The tail-weight parameter. Must be strictly positive. Must be the same
        shape as mu. If tau is None then the default value of tau=1 is used.

    Returns
    -------
    x : float (batch size x 1) Tensor.
        The computed distribution variance values.

    Notes
    -----
    * This code uses two basic formulas:

        var(X) = E(X^2) - (E(X))^2
        var(a*X + b) = a^2 * var(X)

    * The E(X) and E(X^2) are computed using the moment equations given on
    page 764 of [1].

    """
    if tau is None:
        evX = tf.math.sinh(gamma) * 1.35453080648132
        evX2 = (tf.math.cosh(2 * gamma) * 3.0 - 1.0) / 2
    else:
        evX = tf.math.sinh(gamma / tau) * _jones_pewsey_P(1.0 / tau)
        evX2 = (tf.math.cosh(2 * gamma / tau) * _jones_pewsey_P(2.0 / tau) - 1.0) / 2
        
    return tf.math.square(sigma) * (evX2 - tf.math.square(evX))

def percentile_value(mu_pred, sigma_pred, gamma_pred, tau_pred, percentile_frac=0.5):
    """Function to obtain percentile value of the shash distribution."""
    return shash_quantile(
        pr=percentile_frac, mu=mu_pred, sigma=sigma_pred, gamma=gamma_pred, tau=tau_pred
    ).numpy()

In [12]:
   
def compile_model(x_train):

    # First we start with an input layer
    inputs = Input(shape=x_train.shape[1:]) 

    normalizer = tf.keras.layers.Normalization()
    normalizer.adapt(x_train)
    layers = normalizer(inputs)

    layers = Dropout(rate=exp_settings["dropout_rate"],
                     seed=SEED)(layers) 
    
    for hidden, activation, ridge in zip(exp_settings["hiddens"], exp_settings["act_fun"], exp_settings["ridge_param"]):
        layers = Dense(hidden, activation=activation,
                       kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.00, l2=ridge),
                       bias_initializer=tf.keras.initializers.RandomNormal(seed=SEED),
                       kernel_initializer=tf.keras.initializers.RandomNormal(seed=SEED))(layers)


    if exp_settings['network_type'] == 'reg':
        LOSS = 'mae'
        metrics = ['mse',]
        
        output_layer = Dense(1, activation='linear',
                          bias_initializer=tf.keras.initializers.RandomNormal(seed=SEED),
                          kernel_initializer=tf.keras.initializers.RandomNormal(seed=SEED))(layers)
        
    elif exp_settings['network_type'] == 'shash2':
        LOSS = compute_shash_NLL
        metrics = [CustomMAE(name="custom_mae"),
                    InterquartileCapture(name="interquartile_capture"),
                    SignTest(name="sign_test"),
                  ]

        
        y_avg = np.mean(y_train)
        y_std = np.std(y_train)

        mu_z_unit = tf.keras.layers.Dense(
            units=1,
            activation="linear",
            use_bias=True,
            bias_initializer=tf.keras.initializers.RandomNormal(seed=SEED+100),
            kernel_initializer=tf.keras.initializers.RandomNormal(seed=SEED+100),
            name="mu_z_unit",
        )(layers)
        
        mu_unit = tf.keras.layers.Rescaling(
            scale=y_std,
            offset=y_avg,
            name="mu_unit",
        )(mu_z_unit)
        
        # sigma_unit. The network predicts the log of the scaled sigma_z, then
        # the resclaing layer scales it up to log of sigma y, and the custom
        # Exponentiate layer converts it to sigma_y.
        log_sigma_z_unit = tf.keras.layers.Dense(
            units=1,
            activation="linear",
            use_bias=True,
            bias_initializer=tf.keras.initializers.Zeros(),
            kernel_initializer=tf.keras.initializers.Zeros(),
            name="log_sigma_z_unit",
        )(layers)

        log_sigma_unit = tf.keras.layers.Rescaling(
            scale=1.0,
            offset=np.log(y_std),
            name="log_sigma_unit",
        )(log_sigma_z_unit)

        sigma_unit = Exponentiate(
            name="sigma_unit",
        )(log_sigma_unit)
        
        output_layer = tf.keras.layers.concatenate([mu_unit, sigma_unit], axis=1)
        
    else:
        raise NotImpletementedError('no such network_type')
        
    # Constructing the model
    model = Model(inputs, output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=exp_settings["learning_rate"]), 
                  loss=LOSS, 
                  metrics=metrics,
                 )
        
        
    model.summary()
    
    return model


In [13]:
def get_gradients(inputs, top_pred_idx=None):
    """Computes the gradients of outputs w.r.t input image.

    Args:
        inputs: 2D/3D/4D matrix of samples
        top_pred_idx: (optional) Predicted label for the x_data
                      if classification problem. If regression,
                      do not include.

    Returns:
        Gradients of the predictions w.r.t img_input
    """
    inputs = tf.cast(inputs, tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(inputs)
        
        # Run the forward pass of the layer and record operations
        # on GradientTape.
        preds = model(inputs, training=False)  
        
        # For classification, grab the top class
        if top_pred_idx is not None:
            preds = preds[:, top_pred_idx]
        
    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.        
    grads = tape.gradient(preds, inputs)
    return grads

def get_integrated_gradients(inputs, baseline=None, num_steps=50, top_pred_idx=None):
    """Computes Integrated Gradients for a prediction.

    Args:
        inputs (ndarray): 2D/3D/4D matrix of samples
        baseline (ndarray): The baseline image to start with for interpolation
        num_steps: Number of interpolation steps between the baseline
            and the input used in the computation of integrated gradients. These
            steps along determine the integral approximation error. By default,
            num_steps is set to 50.
        top_pred_idx: (optional) Predicted label for the x_data
                      if classification problem. If regression,
                      do not include.            

    Returns:
        Integrated gradients w.r.t input image
    """
    # If baseline is not provided, start with zeros
    # having same size as the input image.
    if baseline is None:
        input_size = np.shape(inputs)[1:]
        baseline = np.zeros(input_size).astype(np.float32)
    else:
        baseline = baseline.astype(np.float32)

    # 1. Do interpolation.
    inputs = inputs.astype(np.float32)
    interpolated_inputs = [
        baseline + (step / num_steps) * (inputs - baseline)
        for step in range(num_steps + 1)
    ]
    interpolated_inputs = np.array(interpolated_inputs).astype(np.float32)

    # 3. Get the gradients
    grads = []
    for i, x_data in enumerate(interpolated_inputs):
        grad = get_gradients(x_data, top_pred_idx=top_pred_idx)
        grads.append(grad[0])
    grads = tf.convert_to_tensor(grads, dtype=tf.float32)

    # 4. Approximate the integral using the trapezoidal rule
    grads = (grads[:-1] + grads[1:]) / 2.0
    avg_grads = tf.reduce_mean(grads, axis=0)

    # 5. Calculate integrated gradients and return
    integrated_grads = (inputs - baseline) * avg_grads
    return integrated_grads

def random_baseline_integrated_gradients(inputs, num_steps=50, num_runs=5, top_pred_idx=None):
    """Generates a number of random baseline images.

    Args:
        inputs (ndarray): 2D/3D/4D matrix of samples
        num_steps: Number of interpolation steps between the baseline
            and the input used in the computation of integrated gradients. These
            steps along determine the integral approximation error. By default,
            num_steps is set to 50.
        num_runs: number of baseline images to generate
        top_pred_idx: (optional) Predicted label for the x_data
                      if classification problem. If regression,
                      do not include.      

    Returns:
        Averaged integrated gradients for `num_runs` baseline images
    """
    # 1. List to keep track of Integrated Gradients (IG) for all the images
    integrated_grads = []

    # 2. Get the integrated gradients for all the baselines
    for run in range(num_runs):
        baseline = np.zeros(np.shape(inputs)[1:])
        for i in np.arange(0,np.shape(random_baseline)[0]):
            j = np.random.choice(np.arange(0,np.shape(inputs)[0]))
            baseline[i] = inputs[j,i]

        igrads = get_integrated_gradients(
            inputs=inputs,
            baseline=baseline,
            num_steps=num_steps,
        )
        integrated_grads.append(igrads)

    # 3. Return the average integrated gradients for the image
    integrated_grads = tf.convert_to_tensor(integrated_grads)
    return tf.reduce_mean(integrated_grads, axis=0)



## Train the network

In [ ]:
rng = np.random.default_rng(exp_settings["rng_seed"])

for iloop in np.arange(exp_settings['n_models']):
    SEED = rng.integers(low=1_000,high=10_000,size=1)[0]    
    tf.random.set_seed(SEED)
    np.random.seed(SEED)

    train_members = rng.choice(ALL_MEMBERS, size=N_TRAIN, replace=False)
    val_members   = rng.choice(np.setdiff1d(ALL_MEMBERS,train_members), size=N_VAL, replace=False)
    test_members  = rng.choice(np.setdiff1d(ALL_MEMBERS,np.append(train_members[:],val_members)), size=N_TEST, replace=False)
    print(train_members, val_members, test_members)

    x_train, x_val, x_test, y_train, y_val, y_test, y_yrs_train, y_yrs_val, y_yrs_test, target_years, map_shape = get_cmip_data()

    tf.keras.backend.clear_session()            
    
#     # make onehot vectors for training
    if exp_settings["network_type"] == 'shash2':
        onehot_train = np.zeros((x_train.shape[0],2))
        onehot_train[:,0] = y_train.astype('float32')
        onehot_val = np.zeros((x_val.shape[0],2))    
        onehot_val[:,0] = y_val.astype('float32')
        onehot_test = np.zeros((x_test.shape[0],2))    
        onehot_test[:,0] = y_test.astype('float32')
    else:
        onehot_train = np.copy(y_train)
        onehot_val = np.copy(y_val)
        onehot_test = np.copy(y_test)
        
    model = compile_model(x_train)
    history = model.fit(x_train, onehot_train, 
                        epochs=exp_settings['n_epochs'], 
                        verbose=exp_settings['verbosity'],
                        batch_size = exp_settings['batch_size'], 
                        shuffle=True,
                        validation_data=[x_val, onehot_val],
                        callbacks=[early_stopping,],
                       )
    #----------------------------------------
    # save the tensorflow model
    model_name = (exp_settings["exp_name"] + '_' + 
                  exp_settings["network_type"] + 
                  '_rng' + str(exp_settings["rng_seed"]) + 
                  '_seed' + str(SEED)
                 )
    if exp_settings["save_model"]:
        save_tf_model(model, model_name)
    
    #----------------------------------------
    if exp_settings["show_plots"]:
        plt.plot(history.history['loss'], label='loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.legend()
        plt.show()
        
        

[9 0 7 5 1 6 3] [8 4 2] []
tas_Amon_historical_ssp370_CanESM5_r1-10_ncecat_ann_mean_2pt5degree.nc
TARGET_YEAR = 2011
tas_Amon_historical_ssp370_GISS-E2-1-G_r1-10_ncecat_ann_mean_2pt5degree.nc
TARGET_YEAR = 2029
tas_Amon_historical_ssp370_IPSL-CM6A-LR_r1-10_ncecat_ann_mean_2pt5degree.nc
TARGET_YEAR = 2020
tas_Amon_historical_ssp370_UKESM1-0-LL_r1-10_ncecat_ann_mean_2pt5degree.nc
TARGET_YEAR = 2024
tas_Amon_historical_ssp370_ACCESS-ESM1-5_r1-10_ncecat_ann_mean_2pt5degree.nc
TARGET_YEAR = 2035
---------------------------
data_train.shape = (35, 131, 72, 144)
data_val.shape = (15, 131, 72, 144)
data_test.shape = (0, 131, 72, 144)
(4585, 10368) (4585,) (4585,)
(1965, 10368) (1965,) (1965,)
(0, 10368) (0,) (0,)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10368)]           0         
                                                                 
 norma

## Make loss plots following training

In [ ]:
def plot_metrics(history,metric):
    
    imin = np.argmin(history.history['val_loss'])
    
    plt.plot(history.history[metric], label='training')
    plt.plot(history.history['val_' + metric], label='validation')
    plt.title(metric)
    plt.axvline(x=imin, linewidth=.5, color='gray',alpha=.5)
    plt.legend()

if exp_settings["network_type"] == 'shash2':
    imin = len(history.history['custom_mae'])#np.argmin(history.history['val_loss'])

    plt.subplots(figsize=(20,4))
    plt.subplot(1,4,1)
    plot_metrics(history,'loss')
    plt.ylim(0,10.)

    plt.subplot(1,4,2)
    plot_metrics(history,'custom_mae')
    plt.ylim(0,10)

    plt.subplot(1,4,3)
    plot_metrics(history,'interquartile_capture')

    plt.subplot(1,4,4)
    plot_metrics(history,'sign_test')

    plt.show()


In [ ]:
if exp_settings['network_type'] == "shash2":
    top_pred_idx = 0
else:
    top_pred_idx = None
    
YEARS_UNIQUE = np.unique(y_yrs_train)
predict_train = model.predict(x_train)[:,top_pred_idx]
predict_val = model.predict(x_val)[:,top_pred_idx]
mae = np.mean(np.abs(predict_val-y_val[:]))

In [ ]:
clr = ('tab:purple','tab:orange', 'tab:pink', 'tab:green', 'gold', 'violet','cornflowerblue','darkorange')
#--------------------------------
plt.subplots(1,2,figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(y_train, predict_train,'.',color='gray',alpha=.5, label='training')
plt.plot(y_val, predict_val,'.', label='validation')
plt.plot(y_val,y_val,'--',color='fuchsia')
plt.axvline(x=0,color='gray',linewidth=1)
plt.axhline(y=0,color='gray',linewidth=1)
plt.title('Validation MAE = ' + str(mae.round(2)) + ' years')
plt.xlabel('true number of years until target is reached')
plt.ylabel('predicted number of years until target is reached')
plt.legend()


plt.subplot(1,2,2)
plt.plot(y_yrs_train, predict_train,'.',color='gray',alpha=.5, label='training')
plt.xlabel('year of map')
plt.ylabel('predicted number of years until target is reached')
plt.axhline(y=0, color='gray', linewidth=1)

predict_val_mat = predict_val.reshape(N_GCMS,N_VAL,len(YEARS_UNIQUE))
for i in np.arange(0,predict_val_mat.shape[0]):
    plt.plot(YEARS_UNIQUE, predict_val_mat[i,:,:].swapaxes(1,0),'.', label='validation', color=clr[i])
    plt.axvline(x=target_years[i],linestyle='--',color=clr[i])
if IN_COLAB==False:
    pass
    # plt.savefig('figures/initial_result_seed' + str(SEED) + '.png', dpi=savefig_dpi)
plt.show()

## Explainability via Input * Gradient and Integrated Gradients
We will use two attribution explainaiblity methods called Input * Gradient and Integrated Gradients to make heatmaps of regions of the input that act as explanations for the network's prediction.

* https://keras.io/examples/vision/integrated_gradients/
* https://distill.pub/2020/attribution-baselines/

In [ ]:
def plot_map(x, clim=None, title=None, text=None, cmap='RdGy'):
    plt.pcolor(x,
               cmap=cmap,
              )
    plt.clim(clim)
    plt.colorbar()
    plt.title(title,fontsize=15,loc='right')    
    plt.yticks([])
    plt.xticks([])
    
    plt.text(0.01, 1.0, text, fontfamily='monospace', fontsize='small', va='bottom',transform=plt.gca().transAxes)

In [ ]:
#=========================================
# Define the samples you want to explain
rng = np.random.default_rng(45)
isubsample = rng.choice(np.arange(0,x_val.shape[0]),
                        size = 500,
                        replace = False,
                       )

inputs = np.copy(x_val[isubsample,:])
targets = np.copy(y_val[isubsample])
yrs = np.copy(y_yrs_val[isubsample])
preds = model.predict(inputs)

#=========================================
#---------------------------------------
# Gradient x Input
#---------------------------------------
# compute the multiplication of gradient * inputs
# and reshape into a map of latitude x longitude

grads = get_gradients(inputs,top_pred_idx).numpy()
grad_x_input = grads * inputs
grad_x_input = grad_x_input.reshape((len(targets),map_shape[0],map_shape[1]))
print(np.shape(grad_x_input))

#---------------------------------------
# Integrated Gradients
#---------------------------------------
baseline_mean = np.mean(x_train,axis=0)*0.    
print('shape(baseline_mean) = ' + str(np.shape(baseline_mean)))
print('model.predict(baseline_mean) = ' + str(model.predict(baseline_mean[np.newaxis,:])))

igrad = get_integrated_gradients(inputs, baseline=baseline_mean,top_pred_idx=top_pred_idx)
integrated_gradients = igrad.numpy().reshape((len(targets),map_shape[0],map_shape[1]))
print(np.shape(integrated_gradients))

In [ ]:

# plot the resulting heatmaps for a subset of samples
# based on their label
plot_list = (60, 40, 20, 10, 0)
NCOL = 4
plt.subplots(len(plot_list),NCOL,figsize=(35,5*len(plot_list)))

for irow,min_range in enumerate(plot_list):
        
    max_range = min_range + 5
    isamples = np.where((targets >= min_range) & (targets <= max_range))[0]
    igrad_mean = np.mean(integrated_gradients[isamples,:,:],axis=0)
    grad_x_input_mean = np.mean(grad_x_input[isamples,:,:],axis=0)
    grad_mean = np.mean(grads[isamples,:],axis=0).reshape((map_shape[0],map_shape[1]))
    x_inputs_mean = np.mean(inputs[isamples,:],axis=0).reshape((map_shape[0],map_shape[1]))
    x_inputs_mean = x_inputs_mean - baseline_mean.reshape((map_shape[0],map_shape[1]))
    #------------------------------------------------------------------
    
    text = (
            "\n"
            + f"  label_range    = {min_range}-{max_range} yrs.\n"                    
            + f"  n_samples      = {len(isamples)}\n"
    )    
    #------------------------------------------------------------------    
    
    # plot average input map
    plt.subplot(len(plot_list),NCOL,irow*NCOL+1)
    plot_map(x_inputs_mean, 
             text=text,
             clim=(-5,5),
             cmap='RdBu_r',
             title = 'Temperature anomaly from Baseline',
            )
    #------------------------------------------------------------------
    # plot explainability of gradient (saliency)
    plt.subplot(len(plot_list),NCOL,irow*NCOL+2)
    plot_map(grad_mean, 
             text=text,             
             clim=(-0.02, .02), 
             title = 'Gradient (Saliency)',
            )
    
    #------------------------------------------------------------------
    # plot explainability of input x gradient
    plt.subplot(len(plot_list),NCOL,irow*NCOL+3)
    plot_map(grad_x_input_mean, 
             text=text,
             clim=(-.02,.02),
             title = 'Gradient x Input',
            )

    #------------------------------------------------------------------
    # plot explainability of integrated gradients
    plt.subplot(len(plot_list),NCOL,irow*NCOL+4)
    plot_map(igrad_mean, 
             text=text,             
             clim=(-.02,.02), 
             title = 'Integrated Gradients',
            )

plt.tight_layout()   
if IN_COLAB==False:
    pass
    # plt.savefig('figures/xai_grid_' + str(min_range) +'-' + str(max_range) + '_baseline_' + str(BASELINE) + '.png', dpi=savefig_dpi)
plt.show()



## Predict observations

In [ ]:
import scipy.stats as stats

if exp_settings["anomalies"]:
    x_obs_predict = np.nan_to_num(x_obs,0.)
y_predict_obs = model.predict(x_obs_predict)[:,top_pred_idx]

iy = np.where(da_obs['time.year'].values >= 2011)[0]
x = da_obs['time.year'].values[iy]
y = y_predict_obs[iy]
linear_model = stats.linregress(x=x,y=y)

#--------------------------------
i_year = np.where(y_predict_obs < 0)[0]
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(da_obs['time.year'], y_predict_obs, '.r')
plt.plot(x, linear_model.slope*x+linear_model.intercept, '--k', alpha=.5, linewidth=2)
plt.xlabel('year of map')
plt.ylabel('predicted number of years \nuntil target is reached')
plt.title('Observations Target Year = ' + 
          str(np.round(2021+y_predict_obs[-1],1)) +
          ' (' + str(y_predict_obs[-1].round()) + ' years)'+
          '\n slope = ' + str(linear_model.slope.round(2)) 
         )
plt.xlim(1970,2025)
# plt.ylim(-20,40)
plt.axhline(y=0,color='gray')

#--------------------------------
plt.subplot(1,2,2)
global_mean_obs.plot(linewidth=2,label='data',color="tab:orange")
plt.title('Observations')

plt.tight_layout()
plt.show()


In [ ]:
grads_obs = get_gradients(x_obs[-1:,:],top_pred_idx=top_pred_idx).numpy()*x_obs[-1:,:]
print('np.shape(grads_obs) = ' + str(np.shape(grads_obs)))
grads_obs_mean = np.mean(grads_obs,axis=0)
plt.figure(figsize=(10,4))
plot_map(grads_obs_mean.reshape((map_shape[0],map_shape[1])), 
         clim = (-.02,.02),
         title = 'Observations - Gradient x Input',
        )

In [ ]:
import scipy.stats as stats
import seaborn as sns

if exp_settings["network_type"] == 'shash2':

    clr_choice = 'orange'
    y_predict_obs = model.predict(x_obs_predict)

    iy = np.where(da_obs['time.year'].values >= 2011)[0]
    x = da_obs['time.year'].values[iy]
    y = y_predict_obs[iy,0]
    linear_model = stats.linregress(x=x,y=y)

    #--------------------------------
    shash_incs = np.arange(-80,80,1)
    mu_pred = y_predict_obs[:,0]
    sigma_pred = y_predict_obs[:,1]
    gamma_pred = np.zeros(mu_pred.shape) #y_predict_obs[:,2]#np.zeros(mu_pred.shape)
    tau_pred = np.ones(mu_pred.shape)
    shash_perc_low = percentile_value(mu_pred, sigma_pred, gamma_pred, tau_pred, percentile_frac=.25)   
    shash_perc_high = percentile_value(mu_pred, sigma_pred, gamma_pred, tau_pred, percentile_frac=.75)
    shash_perc_med = percentile_value(mu_pred, sigma_pred, gamma_pred, tau_pred, percentile_frac=.5)
    shash_cpd = shash_prob(shash_incs,mu_pred[-1],sigma_pred[-1],gamma_pred[-1],tau_pred[-1])

    print('2021 prediction = ' + str(mu_pred[-1]) + ' (' + str(shash_perc_low[-1]) + ' to ' + str(shash_perc_high[-1]) + ')')

    y_predict_obs = mu_pred #shash_perc_med #shash_mean(mu_pred, sigma_pred, gamma_pred, tau_pred)#y_predict_obs[:,0]

    #------------------------------------------------------------
    ax = plt.subplots(1,2,figsize=(16,4))
    years = np.arange(1850,2022)

    plt.subplot(1,2,1)
    for iyear in np.arange(0,y_predict_obs.shape[0]):
        min_val = shash_perc_low[iyear]
        max_val = shash_perc_high[iyear]

        if(years[iyear]==2021):
            clr = clr_choice
        else:
            clr = 'gray'
        plt.plot((years[iyear],years[iyear]),(min_val, max_val),
                 linestyle='-',
                 linewidth=4,
                 color=clr,
                )

    plt.plot(x,x*linear_model.slope+linear_model.intercept,'--', color='black')

    plt.xlim(1977.5,2023)    
    plt.ylim(0,57)
    plt.ylabel('years until target')
    plt.xlabel('year')
    plt.title('Observations predictions under SSP337 (shash)\n slope=' + str(linear_model.slope.round(2)))

    plt.subplot(1,2,2)
    plt.plot(shash_incs,shash_cpd,
             linewidth=5,
             color=clr_choice,
            )

    k = np.argmin(np.abs(shash_perc_low[-1]-shash_incs))
    plt.plot((shash_perc_low[-1],shash_perc_low[-1]),(0,shash_cpd[k]),'--',color=clr_choice)
    k = np.argmin(np.abs(shash_perc_high[-1]-shash_incs))
    plt.plot((shash_perc_high[-1],shash_perc_high[-1]),(0,shash_cpd[k]),'--',color=clr_choice)

    plt.xlabel('years until target')
    plt.title('Predictions for Observations\nYear = 2021')
    plt.xlim(-10,40)
    plt.tight_layout()
    plt.show()